In [ ]:
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from IPython.display import clear_output

import pickle
import keras
from keras.optimizers import SGD
from keras.layers import Input, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout
from keras import Model
from keras.layers import GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.applications.mobilenet import decode_predictions
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet import preprocess_input

## Load data

In [ ]:
which_data = '_smaller' #choice between '', '_smaller', '_smallest'

size_image = 256 # choice between 128 and 256

In [ ]:
datagen = ImageDataGenerator(featurewise_center = False,
                             preprocessing_function=preprocess_input,
                            rescale = 1/256)

#import data
train_in = datagen.flow_from_directory('../../data/Building_labeled_train_data'+str(which_data),
#                                       color_mode="rgb",
                                       target_size = (size_image, size_image),
#                                        batch_size=32,
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42)

test_in = datagen.flow_from_directory('../../data/Building_labeled_test_data'+str(which_data),
#                                       color_mode="rgb",
                                      target_size = (size_image, size_image),
#                                        batch_size=32,
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42)

In [ ]:
X_train, y_train = train_in.next()
X_test, y_test = test_in.next()

In [ ]:
print ('X_train: ', X_train.shape,
       '\ny_train: ', y_train.shape,
       '\nX_test: ', X_test.shape,
       '\ny_test: ', y_test.shape)

In [ ]:
x,y = train_in.next()
for i in range(0,5):
    image = x[i] 
    plt.imshow(image)
    plt.show()

In [ ]:
input_shape = X_train[0].shape #the input shape should not include batch size
n_classes = y_train[0].shape[0]

## Basic cnn model

In [ ]:
def basic_cnn_model(input_shape):
    X_input = Input((input_shape))
    print('made input layer: ', X_input.shape)
    
    #conv
    X = Conv2D(16, (3,3), name='conv0')(X_input)
    print('after conv2d: ', X.shape)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    print('after BN: ', X.shape)
    X = Activation('relu')(X)
    print('after activation: ', X.shape)
    X = MaxPooling2D((2), name='max_pool0')(X)
    print('after maxpool2d: ', X.shape)
    
    X = Conv2D(25, (6,6), name='conv1')(X)
    print('after conv2d: ', X.shape)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    print('after BN: ', X.shape)
    X = Activation('relu')(X)
    print('after activation: ', X.shape)
    X = MaxPooling2D((2), name='max_pool1')(X)
    print('after maxpool2d: ', X.shape)
    
    
    
    #dropout
    X = Dropout(0.5)(X)
    print('after dropout: ', X.shape)
    
    #rest
    X = Flatten()(X)
    print('after flatten: ', X.shape)
    
    #add one dense layer
    X = Dense(150, activation='relu', name='dense')(X)
    print('after dense: ', X.shape)
    
    X = Dropout(0.5)(X)
    
    #final prediction
    X = Dense(n_classes, activation='softmax', name='final_dense')(X)
    print('after dense: ', X.shape)
    
    model = Model(inputs = X_input, outputs = X,  name='basic_cnn')
    #here we are only building the model, that starting from X_input leads to (the last
    #X) through all the layers
    return model

In [ ]:
# clear_session()

In [ ]:
cnn_model = basic_cnn_model(input_shape)

In [ ]:
early_stop = EarlyStopping(
    monitor='accuracy', min_delta=0.01, patience=10, verbose=1, mode='max',
    baseline=None, restore_best_weights=False)

In [ ]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
#with multiprocessing: tot time = 
#without: tot time = 


# history = cnn_model.fit(X_train_sub, y_train_onehot_sub, batch_size=32, epochs=300,
#              validation_split=0.2)

# train_set = train_in[:train_in.s]


history = cnn_model.fit_generator(train_in, epochs=5, validation_data=test_in, 
                                  callbacks=[early_stop],
                                 validation_freq=1,
                                 use_multiprocessing=True
                                 )


# history = cnn_model.fit_generator(train_in, epochs=5, 
#                                   callbacks=[early_stop], 
#                                   use_multiprocessing=True)

In [ ]:
%timeit
### but this trains only on 1 batch of the data from the generator

# history = cnn_model.fit(X_train, y_train, epochs=5, 
#                         callbacks=[early_stop],
# #                         val_split=0.2,
#                        use_multiprocessing=True)

In [ ]:
## print('initial accuracy on train: ',  cnn_model.history.history['accuracy'][0])
print('final accuracy on train: ',  cnn_model.history.history['accuracy'][-1])

print('initial accuracy on val: ',  cnn_model.history.history['val_accuracy'][0])
print('final accuracy on val: ',  cnn_model.history.history['val_accuracy'][-1])

basic model (10 epochs): 
tr 0.73 -> 0.86
val 0.25 -> 0.22

basic model (20 epochs):
tr 0.86 -> 0.92
val 0.23 -> 0.28


adding 1 more conv layer and 1 more dense layer (20 epochs):
seems not to improve (stopped)

with pretrained mobilenet (10 epochs):
tr: 0.24 -> 0.41
val: 0.28 -> 0.33


### some notes (with toy data)

* inital configuration:
conv2d
batchnormalization
activation
maxpooling
flatten
dense

gave training accuracy 0.18 (1st epoch) -> 1 (last epoch), val accuracy 0.24 -> 0.37

So the train goes well but not the validation: need to regularize and/or have more data

* adding a dropout (after maxpooling): not much improvement
* adding dropout AND training on more data: tr: 0.17 -> 0.98, val 0.15 -> 0.4
* without dropout but more data: tr 0.2 -> 1, val 0.25 -> 0.42

the higher dropout the better improvement (0.5 better than 0.2)

* adding a dense layer before the final layer: not improved

* adding another conv2d layer (with batch, act, maxpooling): tr same, val 0.16 -> 0.4
but the val loss behaves better

* with more conv filters: tr same, val 0.12 -> 0.42

* with larger batch size: tr same, val 0.13 -> 0.43

* with more epochs: tr same, val 0.12 -> 0.46


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
score = cnn_model.evaluate(X_test_sub, y_test_onehot_sub, verbose=0)

In [ ]:

print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Pretrained net

In [ ]:
clear_session()

In [ ]:
base_model_vgg128 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model_vgg256 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
#import few models

size_image=256
# base_model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(size_image, size_image, 3))

base_model_inceptionv3 = InceptionV3(weights='imagenet', 
                                     include_top=False, 
                                     input_shape=(size_image, size_image, 3))

base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(size_image, size_image, 3))

base_model_xception = Xception(weights='imagenet', include_top=False, input_shape=(size_image, size_image, 3))

In [ ]:
base_model_vgg19.summary()

In [ ]:
base_model = base_model_vgg256

In [ ]:
#if I need to save it to use it on the cluster

models_list = [base_model_vgg128, base_model_vgg256, base_model_inceptionv3, 
               base_model_vgg19, base_model_xception]
models_names = ['base_model_vgg128', 'base_model_vgg256', 'base_model_inceptionv3_256',
               'base_model_vgg19_256', 'base_model_xception_256']

for model, model_name in zip(models_list, models_names):
    with open(f'{model_name}.pickle', 'wb') as f:
        pickle.dump(model, f)

In [ ]:
with open('base_model_inceptionv3_256.pickle', 'wb') as f:
        pickle.dump(base_model_inceptionv3, f)

In [ ]:
base_model.summary()

In [ ]:
# add last layers


x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(n_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# callbacks
early_stop = EarlyStopping(
    monitor='loss', 
    min_delta=0.01, 
    patience=15, 
    verbose=1, mode='auto')


In [ ]:

class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.loss = []
        self.val_loss = []
        self.accuracy = []
        self.val_accuracy = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.accuracy.append(logs.get('acc'))
        self.val_accuracy.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.loss, label="loss")
        ax1.plot(self.x, self.val_loss, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.accuracy, label="accuracy")
        ax2.plot(self.x, self.val_accuracy, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

In [ ]:
#freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

# for layer in model.layers:
#     print(layer, layer.trainable)

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy',
             metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
mn_history = model.fit_generator(train_in, epochs=5,
#                                  validation_data=test_in, 
                                  callbacks=[early_stop, plot, ],
#                                  validation_freq=1,
                                 use_multiprocessing=True
                                 )


In [ ]:
print('initial accuracy on train: ', mn_history.history['accuracy'][0])
print('final accuracy on train: ', mn_history.history['accuracy'][-1])

print('initial accuracy on val: ', mn_history.history['val_accuracy'][0])
print('final accuracy on val: ', mn_history.history['val_accuracy'][-1])

#_inception v2 resnet
#_5 epochs
#_128x128
initial accuracy on train:  0.2320909
final accuracy on train:  0.36954546
initial accuracy on val:  0.26877760887145996
final accuracy on val:  0.2989690601825714


#_inception v2 resnet
#_5 epochs
#_256x256
initial accuracy on train:  0.28054544
final accuracy on train:  0.4851818
initial accuracy on val:  0.3343151807785034
final accuracy on val:  0.38954344391822815



In [ ]:
# Plot training & validation accuracy values
plt.plot(mn_history.history['accuracy'])
plt.plot(mn_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(mn_history.history['loss'])
plt.plot(mn_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()